# Client-Side Storage

Client Side storage consists of JavaScript API's that let us store data on the client (like user's machine) and then retrieve it when needed. So it can:  
* Personalize the site preferences (dark mode, font size)
* Previous site activity (Shopping cart, remember if user was already logged in)
* Saving data and assets locally to load quicker
* Saving documents

But client-size AND server-side storage can be used together. So like downloading music files, and then store them in a client-side database.

## Old School: Cookies

In the early days of the web, sites used cookies to store information. So we will not be learning how to use cookies. Look here for more info: <a href="https://developer.mozilla.org/en-US/docs/Web/HTTP/Cookies">Cookies</a>

## New School: Web Storage and IndexedDB

1. Web Storage API allows us to store and retrieve small data items consistening of a name and a value. Useful for data like user's name, if they are logged in, color for the background of the screen, etc.
2. IndexedDB API provides the browser with a complete database system to store complex data. Store complete set of records, audo files, video files, etc.
3. Cache API stores HTTP responses to specific requests, and can store stuff like website assets offline to use without a network connection. We will not cover this in great detail.

## Storing Simple Data - Web Storage

All web storage is contained within object-like structures inside the browser:
1. `sessionStorage`, which persists data for as long as the browser is open. (Data is lost when the browser is closed)
2. `localStorage`, persists data even if the browser is closed.  
We will use `localStorage` as it is generally more useful.

`localStorage.setItem()` allows use to save a data item in storage, and it takes two parameters: the name of the item, and the value.

Type this in your web page's console:

In [ ]:
localStorage.setItem("name", "YourName");

Now we use `localStorage.getItem()` which only takes one parameter, the name of a data item we want to retrieve. It then returns the item's value:

In [ ]:
let myName = localStorage.getItem("name");

The `localStorage.removeItem()` method takes one parameter, the name of the item. This will remove the item out of the web storage:

In [ ]:
localStorage.removeItem("name");
myName = localStorage.getItem("name");
myName;

What will myName return?

Every domain has its OWN storage of web data items. If you try to get the item you set in another page, it won't be there, or won't be refer to the same item.

Let's try to make a very simple test of this:

In [ ]:
<body>
    <h1></h1>
    <p>Type Name Below:</p>
    <input type="text"></input>
    <button>Submit</button>
    <script>
        let h = document.querySelector("h1");
        let inp = document.querySelector('input');
        let button = document.querySelector('button');
        let myName = localStorage.getItem("name");
        if (myName === null){
            h.textContent = "No one is saved";
        }
        else
        {
            h.textContent = myName;
        }
    
        button.addEventListener('click', (e) => {
            e.preventDefault();
            localStorage.setItem('name', inp.value);
        });
    
    </script>
</body>

Try it out!

## Storing Complex Data - IndexedDB

This is a **Complete Database System** available in the browser to store complex related data. This API lets us create a database, and create object stores within the data base. Object stores are like tables in a **Relational Database**, and each object store can contain a number of objects.

We will start by creating a Note Storing website.  
Each note will have a title and body of text to go along with it. You can also delete each individual note.

In [1]:
<head>
    <style>

    </style>
</head>

<body>
    <h1 id="HeadTitle">Note Taking App</h1>
    <script>
        let db;
    </script>
</body>

Note Taking App

`let db;`  
We are declraing a variable, `db`, which we will use to represent our database.

In [ ]:
<head>
    <style>

    </style>
</head>

<body>
    <h1 id="HeadTitle">Note Taking App</h1>
    <script>
        let db;
        const openRequest = window.indexedDB.open("notes_db", 1);
    </script>
</body>

This creates a request to open version `1` of a database called `notes_db`.  
If it does not exist, it will create the database. Database operations are all asynchronous, so this returns a `request` object. We will use event handlers to run code when the request completes, fails, etc.

In [ ]:
<head>
    <style>

    </style>
</head>

<body>
    <h1 id="HeadTitle">Note Taking App</h1>
    <script>
        let db;
        const openRequest = window.indexedDB.open("notes_db", 1);
        openRequest.addEventListener("error", () =>
            console.error("Database failed to open")
        );
        openRequest.addEventListener("success", () => {
            console.log("Database opened successfully");
            db = openRequest.result;
            displayData();
        });
    </script>
</body>

You can guess what the first event listener does.  
In the second event listener, an object representing the opened database becomes available in `openRequest.result`, which allows us to manipulate the database. `displayData()` will be made later on.

Finally, we will do this:

In [ ]:
<head>
    <style>

    </style>
</head>

<body>
    <h1 id="HeadTitle">Note Taking App</h1>
    <script>
        let db;
        const openRequest = window.indexedDB.open("notes_db", 1);
        openRequest.addEventListener("error", () =>
            console.error("Database failed to open")
        );
        openRequest.addEventListener("success", () => {
            console.log("Database opened successfully");
            db = openRequest.result;
            displayData();
        });
        // Set up the database tables if this has not already been done
        openRequest.addEventListener("upgradeneeded", (e) => {
            // Grab a reference to the opened database
            db = e.target.result;
            // Create an objectStore in our database to store notes and an auto-incrementing key
            // An objectStore is similar to a 'table' in a relational database
            const objectStore = db.createObjectStore("notes_os", {
                keyPath: "id",
                autoIncrement: true,
            });
            // Define what data items the objectStore will contain
            objectStore.createIndex("title", "title", { unique: false });
            objectStore.createIndex("body", "body", { unique: false });

            console.log("Database setup complete");
        });
    </script>
</body>

This event listener will run if the database has not been set up, or if there is a bigger version number than the existing stored database.  
This is where we define the schema (structure) of our database. So the set of columns it contains. So we grab the `db` using `e.target.result` which you should understand.  
`IDBDatabse.createObjectStore()` creates a new object store (table) inside our opened database called `"notes_os"`, and has an `autoIncrement` key field called `id`. Which means that each new record will automatically be given an incremented `id` value, so we don't have to set each `record`'s `id` value manually. `id` will be used to uniquely identify records. We also made two other indexes: `"title"` and `"body"`.

If anything confuses you from the explanation above, you will see for yourself later on what this means.

### Adding data to the Database

We will use a form on the page to add in records to the database.  
First we will add `<form>`, `<input>`, and `<button>` elements to the page as a way to submit the form to use as an event handler to `addData()`:

In [ ]:
<head>
    <style>

    </style>
</head>

<body>
    <h1 id="HeadTitle">Note Taking App</h1>
    <h2 id="newNotes">Enter a New Note:</h2>
    <form>
        <p>Note Title</p>
        <input id="title" type="text" required>
        <p>Note Text</p>
        <input id="body" type="text" required>
        <button>Create new note</button>
    </form>

    <script>
        const form = document.querySelector('form');

        let db;
        const openRequest = window.indexedDB.open("notes_db", 1);
        openRequest.addEventListener("error", () =>
            console.error("Database failed to open")
        );
        openRequest.addEventListener("success", () => {
            console.log("Database opened successfully");
            db = openRequest.result;
            displayData();
        });
        // Set up the database tables if this has not already been done
        openRequest.addEventListener("upgradeneeded", (e) => {
            // Grab a reference to the opened database
            db = e.target.result;
            // Create an objectStore in our database to store notes and an auto-incrementing key
            // An objectStore is similar to a 'table' in a relational database
            const objectStore = db.createObjectStore("notes_os", {
                keyPath: "id",
                autoIncrement: true,
            });
            // Define what data items the objectStore will contain
            objectStore.createIndex("title", "title", { unique: false });
            objectStore.createIndex("body", "body", { unique: false });

            console.log("Database setup complete");
        });

        form.addEventListener("submit", addData);
    </script>
</body>

Make sure you understand what I did with the `<form>` and how I added the `form.addEventListener("submit", addData)`. `"submit"` occurs when the `<button>` in the form is pressed, which `"submits"` a form.

Now let's create the `addData()` function:

In [ ]:
<head>
    <style>

    </style>
</head>

<body>
    <h1 id="HeadTitle">Note Taking App</h1>
    <h2 id="newNotes">Enter a New Note:</h2>
    <form>
        <p>Note Title</p>
        <input id="title" type="text" required>
        <p>Note Text</p>
        <input id="body" type="text" required>
        <button>Create new note</button>
    </form>

    <script>
        const form = document.querySelector('form');

        let db;
        const openRequest = window.indexedDB.open("notes_db", 1);
        openRequest.addEventListener("error", () =>
            console.error("Database failed to open")
        );
        openRequest.addEventListener("success", () => {
            console.log("Database opened successfully");
            db = openRequest.result;
            displayData();
        });
        
        // Set up the database tables if this has not already been done
        openRequest.addEventListener("upgradeneeded", (e) => {
            db = e.target.result;
            const objectStore = db.createObjectStore("notes_os", {
                keyPath: "id",
                autoIncrement: true,
            });
            objectStore.createIndex("title", "title", { unique: false });
            objectStore.createIndex("body", "body", { unique: false });
            console.log("Database setup complete");
        });

        // Define the addData() function
        function addData(e) {
            e.preventDefault();
            const newItem = { title: titleInput.value, body: bodyInput.value };
            const transaction = db.transaction(["notes_os"], "readwrite");
            const objectStore = transaction.objectStore("notes_os");
            const addRequest = objectStore.add(newItem);
            addRequest.addEventListener("success", () => {
                titleInput.value = "";
                bodyInput.value = "";
            });
            transaction.addEventListener("complete", () => {
                console.log("Transaction completed: database modification finished.");
                displayData();
            });
            transaction.addEventListener("error", () =>
                console.log("Transaction not opened due to error")
            );
        }

        form.addEventListener("submit", addData);
    </script>
</body>

Let's breakdown each step:
1. `e.preventDefault()` to prevent conventional form submission. This would refresh the page if we forgot.  
2. `const newItem = ...` will create a record to enter into the database. Notice the key and values, and also notice we did not use `id` because it is automatically made for us.
3. We are opening a `readwrite` `"transaction"` against the `notes_os` object store using the `IDBDatabase.transaction()` method. This lets us access the object store so we can do something to it. In this case, add a new record.
4. Now we access the object store using the `IDBTransaction.objectStore()` method, saving the result in the `objectStore` variable.
5. Add the record to the object store by using `objectStore.add(newItem)`, which will create a request object, just like it did when we made an `openRequest` for the database itself.
6. You can read each change event listener as it is self explanatory.

Now, let's actually define how we will display the data, which in itself is complex:

1. When the page loads, we start off with an empty `<ul>`. This will be our `list` reference using the `querySelector`. So we use a while loop and `list.firstChild` to see if there is still an element in the `<ul>`. If there is still an element, we use `list.removeChild(list.firstChild)` to remove it.
2. Then we get a reference to the `notes_os` object store using the same `IDBDatabse.transaction()` and `IDBTransaction.objectStore()`, except the transaction is only `read`, and we are chaining the two methods together.
3. `IDBObjectStore.openCursor()` opens a REQUEST for a cursor. A cursor is used to iterate over the records in an object store. Because this is a request, we use this as an event listener using the "success" event.
4. We get the reference to the cursor itself by doing `const cursor = e.target.result`.
5. If the cursor has a record from the datastore, `if (cursor) {}`, we create a DOM fragment, populate it with the info from the record, and insert it into the page. Then we include a delete button which will run the `deleteItem()` function which we will define later on.
6. At the end of the `if` block, we use `IDBCursor.continue()` to advance the cursor to the next record in the datastore.
7. If there are no more records, the `cursor` will return `undefined`, and so the `else` block will run.

Here is the code:

In [ ]:
<head>
    <style>

    </style>
</head>

<body>
    <h1 id="HeadTitle">Note Taking App</h1>
    <h2 id="newNotes">Enter a New Note:</h2>
    <ul>

    </ul>
    <form>
        <p>Note Title</p>
        <input id="title" type="text" required>
        <p>Note Text</p>
        <input id="body" type="text" required>
        <button>Create new note</button>
    </form>

    <script>
        const form = document.querySelector('form');
        const list = document.querySelector('ul');
        const titleInput = document.querySelector('#title');
        const bodyInput = document.querySelector('#body');
        let db;
        const openRequest = window.indexedDB.open("notes_db", 1);
        openRequest.addEventListener("error", () =>
            console.error("Database failed to open")
        );
        openRequest.addEventListener("success", () => {
            console.log("Database opened successfully");
            db = openRequest.result;
            displayData();
        });
        
        // Set up the database tables if this has not already been done
        openRequest.addEventListener("upgradeneeded", (e) => {
            db = e.target.result;
            const objectStore = db.createObjectStore("notes_os", {
                keyPath: "id",
                autoIncrement: true,
            });
            objectStore.createIndex("title", "title", { unique: false });
            objectStore.createIndex("body", "body", { unique: false });
            console.log("Database setup complete");
        });

        // Define the displayData() function
        function displayData() {
            while (list.firstChild) {
                list.removeChild(list.firstChild);
            }
            const objectStore = db.transaction("notes_os").objectStore("notes_os");
            objectStore.openCursor().addEventListener("success", (e) => {
                const cursor = e.target.result;
                if (cursor) {
                    const listItem = document.createElement("li");
                    const h3 = document.createElement("h3");
                    const para = document.createElement("p");
                    listItem.appendChild(h3);
                    listItem.appendChild(para);
                    list.appendChild(listItem);
                    h3.textContent = cursor.value.title;
                    para.textContent = cursor.value.body;
                    // Store the ID of the data item inside an attribute on the listItem, so we know
                        // which item it corresponds to. This will be useful later when we want to delete items
                    listItem.setAttribute("data-note-id", cursor.value.id);

                    const deleteBtn = document.createElement("button");
                    listItem.appendChild(deleteBtn);
                    deleteBtn.textContent = "Delete";
                    //deleteBtn.addEventListener("click", deleteItem);
                    cursor.continue();
                } else {
                    // Again, if list item is empty, display a 'No notes stored' message
                    if (!list.firstChild) {
                        const listItem = document.createElement("li");
                        listItem.textContent = "No notes stored.";
                        list.appendChild(listItem);
                    }
                    // if there are no more cursor items to iterate through, say so
                    console.log("Notes all displayed");
                }
            });
        }


        // Define the addData() function
        function addData(e) {
            e.preventDefault();
            const newItem = { title: titleInput.value, body: bodyInput.value };
            const transaction = db.transaction(["notes_os"], "readwrite");
            const objectStore = transaction.objectStore("notes_os");
            const addRequest = objectStore.add(newItem);
            addRequest.addEventListener("success", () => {
                titleInput.value = "";
                bodyInput.value = "";
            });
            transaction.addEventListener("complete", () => {
                console.log("Transaction completed: database modification finished.");
                displayData();
            });
            transaction.addEventListener("error", () =>
                console.log("Transaction not opened due to error")
            );
        }


        form.addEventListener("submit", addData);
    </script>
</body>

Run this and make sure it works.  
Take note of the `listItem.setAttribute()` method we called, as we will be using it later!

Let's now implement the `deleteItem` function, but I will explain it first again as usual:

1. Retrieve the `id` and store it in `const noteId` and change turn it into a `Number()`
2. Reference the object store using the same pattern as before, but instead use the `IDBObjectStore.delete()` method, passing in the `id`.
3. We then remove the child node from the parent node.

In [ ]:
<head>
    <style>

    </style>
</head>

<body>
    <h1 id="HeadTitle">Note Taking App</h1>
    <h2 id="newNotes">Enter a New Note:</h2>
    <ul>

    </ul>
    <form>
        <p>Note Title</p>
        <input id="title" type="text" required>
        <p>Note Text</p>
        <input id="body" type="text" required>
        <button>Create new note</button>
    </form>

    <script>
        const form = document.querySelector('form');
        const list = document.querySelector('ul');
        const titleInput = document.querySelector('#title');
        const bodyInput = document.querySelector('#body');
        let db;
        const openRequest = window.indexedDB.open("notes_db", 1);
        openRequest.addEventListener("error", () =>
            console.error("Database failed to open")
        );
        openRequest.addEventListener("success", () => {
            console.log("Database opened successfully");
            db = openRequest.result;
            displayData();
        });
        
        // Set up the database tables if this has not already been done
        openRequest.addEventListener("upgradeneeded", (e) => {
            db = e.target.result;
            const objectStore = db.createObjectStore("notes_os", {
                keyPath: "id",
                autoIncrement: true,
            });
            objectStore.createIndex("title", "title", { unique: false });
            objectStore.createIndex("body", "body", { unique: false });
            console.log("Database setup complete");
        });

        // Define the displayData() function
        function displayData() {
            while (list.firstChild) {
                list.removeChild(list.firstChild);
            }
            const objectStore = db.transaction("notes_os").objectStore("notes_os");
            objectStore.openCursor().addEventListener("success", (e) => {
                const cursor = e.target.result;
                if (cursor) {
                    const listItem = document.createElement("li");
                    const h3 = document.createElement("h3");
                    const para = document.createElement("p");
                    listItem.appendChild(h3);
                    listItem.appendChild(para);
                    list.appendChild(listItem);
                    h3.textContent = cursor.value.title;
                    para.textContent = cursor.value.body;
                    // Store the ID of the data item inside an attribute on the listItem, so we know
                        // which item it corresponds to. This will be useful later when we want to delete items
                    listItem.setAttribute("data-note-id", cursor.value.id);

                    const deleteBtn = document.createElement("button");
                    listItem.appendChild(deleteBtn);
                    deleteBtn.textContent = "Delete";
                    deleteBtn.addEventListener("click", deleteItem);
                    cursor.continue();
                } else {
                    // Again, if list item is empty, display a 'No notes stored' message
                    if (!list.firstChild) {
                        const listItem = document.createElement("li");
                        listItem.textContent = "No notes stored.";
                        list.appendChild(listItem);
                    }
                    // if there are no more cursor items to iterate through, say so
                    console.log("Notes all displayed");
                }
            });
        }


        // Define the addData() function
        function addData(e) {
            e.preventDefault();
            const newItem = { title: titleInput.value, body: bodyInput.value };
            const transaction = db.transaction(["notes_os"], "readwrite");
            const objectStore = transaction.objectStore("notes_os");
            const addRequest = objectStore.add(newItem);
            addRequest.addEventListener("success", () => {
                titleInput.value = "";
                bodyInput.value = "";
            });
            transaction.addEventListener("complete", () => {
                console.log("Transaction completed: database modification finished.");
                displayData();
            });
            transaction.addEventListener("error", () =>
                console.log("Transaction not opened due to error")
            );
        }

        function deleteItem(e) {
            const noteId = Number(e.target.parentNode.getAttribute("data-note-id"));
            const transaction = db.transaction(["notes_os"], "readwrite");
            const objectStore = transaction.objectStore("notes_os");
            const deleteRequest = objectStore.delete(noteId);

            transaction.addEventListener("complete", () => {
                e.target.parentNode.parentNode.removeChild(e.target.parentNode);
                console.log(`Note ${noteId} deleted.`);
                if (!list.firstChild) {
                const listItem = document.createElement("li");
                listItem.textContent = "No notes stored.";
                list.appendChild(listItem);
                }
            });
        }


        form.addEventListener("submit", addData);
    </script>
</body>

### And we are DONE!!

## Video data using IndexedDB